# Big Data Modeling and Management Assigment

Gustavo Brito r20170760 - Mariana Albernaz r20170785 - Marta Santos r20170770 - Miguel Mateus r20170752


## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_ 

#### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions

#### Connection details to the neo4j database
```
Host: rhea.isegi.unl.pt:7474  
Username: neo4j  
Password: F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ 
Connect URL : bolt://rhea.isegi.unl.pt:7687
```


#### Questions


0. __Example Question__ _How many beers does the database contain?_
1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers?
    1. Which `Country` has the most reviews for its beers? 
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV?
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
1. Using Graph Algorithms answer **two** of the following questions:
    1. Which two Countries are most similiar when it comes to their **top 10** most produced Beer styles?
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    1. Which user is the most influential when it comes to reviews made?
    1. Which beer styles are more central when it comes the amount of beers? \
    Note: In case of a tie for the top entity, in terms of metrics outputed from the algorithms, **simply output the first.**
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why?


 Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
##### Groups  

Groups should have 4 to 5 people  
You should register your group on moodle. An email will be going out to everyone with the credentials for the database to use when storing the results.


##### Submission      

Submission of the query results to be done to the group's redis database (explained on the first class, credentials sent via email).  
The following format is expected:
```
    >>> redis.set("0", "358873")
```

This result should be the anwser of the group to question 0

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of May 2nd**

##### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

**Note:**
Remember the Neo4j is a shared database and when creating in-memory graphs please use your group's prefix.  
Ex. Instead of `my-graph` as the name of your graph please use `group0-my-graph`.

In [1]:
import py2neo

username="neo4j"
password="F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ"
host="rhea.isegi.unl.pt"
port="7687"

secure_graph = py2neo.Graph(f"bolt://{username}:{password}@{host}:{port}")
secure_graph.run("MATCH () RETURN count(*)").data()

[{'count(*)': 9647598}]

## 0. Example Question How many beers does the database contain?

In [3]:
secure_graph.run("""
        MATCH (b:Beers)
        RETURN count(b) as total_number_of_beers
""").data()

[{'total_number_of_beers': 358873}]

## 1. How many different countries exist in the database?

In [4]:
secure_graph.run("""
        MATCH (c:Country)
        RETURN count(c) as total_number_of_countries
""").data()

[{'total_number_of_countries': 200}]

## 2A. Which Beer has the most reviews?

In [5]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]->(b:Beers)
 return b.name, count(r) as number_of_reviews
 order by number_of_reviews desc
 limit 1
""").data()

[{'b.name': 'IPA', 'number_of_reviews': 31387}]

## 2B. Which Brewery has the most reviews for its beers?

In [6]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]-(:Beers)-[:BREWED_AT]-(br:Breweries)
 return br.name, count(r) as number_of_reviews
 order by number_of_reviews desc
 limit 1
""").data()

[{'br.name': 'Sierra Nevada Brewing Co.', 'number_of_reviews': 175161}]

## 2C. Which Country has the most reviews for its beers?

In [3]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]-(:Beers)-[:BREWED_AT]-(:Breweries)-[:FROM]-(c:Country)
 return c.country_digit, count(r) as number_of_reviews
 order by number_of_reviews desc
 limit 1
""").data()

[{'c.country_digit': 'US', 'number_of_reviews': 7524410}]

## 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [8]:
secure_graph.run("""
 match (:Username{user_name: 'CTJman'})-[:MADE]-(:Reviews)-[:ABOUT]-(:Beers)-[:ABOUT]-(r2:Reviews)-[:MADE]-(u2:Username)
 return u2.user_name, count(r2) as shared_reviews
 order by shared_reviews desc
 limit 5
""").data()

[{'u2.user_name': 'acurtis', 'shared_reviews': 1428},
 {'u2.user_name': 'Texasfan549', 'shared_reviews': 1257},
 {'u2.user_name': 'kjkinsey', 'shared_reviews': 1205},
 {'u2.user_name': 'oline73', 'shared_reviews': 1191},
 {'u2.user_name': 'chippo33', 'shared_reviews': 1161}]

## 4. Which Portuguese brewery has the most beers?

In [9]:
secure_graph.run("""
 match (b:Beers)-[:BREWED_AT]-(br:Breweries)-[:FROM]-(:Country{country_digit: 'PT'})
 return br.name, count(b) as number_of_beers
 order by number_of_beers desc
 limit 1
""").data()

[{'br.name': 'Dois Corvos Cervejeira', 'number_of_beers': 40}]

## 5. From those beers (the ones returned from the previous question), which has the most reviews?

In [10]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]-(b:Beers)-[:BREWED_AT]-(br:Breweries{name: 'Dois Corvos Cervejeira'})
 return b.name, count(r) as number_of_reviews
 order by number_of_reviews desc
 limit 1
""").data()

[{'b.name': 'Finisterra', 'number_of_reviews': 10}]

## 6. On average how many different beer styles does each brewery produce?

In [11]:
secure_graph.run("""
 match (br:Breweries)-[:BREWED_AT]-(b:Beers)-[:OF_TYPE]-(s:Style)
 with br, count(DISTINCT s) as styles
 return avg(styles) as avg_different_styles
""").data()

[{'avg_different_styles': 10.595992516144603}]

## 7. Which brewery produces the strongest beers according to ABV?

To see which brewery produces the strongest beers according to ABV, we choose to select the one that produces, on average, beers with the highest ABV.

In [12]:
secure_graph.run("""
 match (b:Beers)-[:BREWED_AT]-(br:Breweries)
 where b.abv <> 'Unknown'
 return br.name, avg(toFloat(b.abv)) as avg_abv
 order by avg_abv desc
 limit 1
""").data()

[{'br.name': '1648 Brewing Company Ltd', 'avg_abv': 25.57777777777778}]

## 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

To suggest a beer style, we choose to select the one that had, on average, the highest combined scores on smell (aroma) and look (appearance), across all reviews.

In [13]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]-(:Beers)-[:OF_TYPE]-(s:Style)
 where  r.look <> 'Unknown' AND r.smell <> 'Unknown'
 return s.name AS style, avg(toFloat(r.look)) + avg(toFloat(r.smell)) AS FinalScore, avg(toFloat(r.look)) as appearance, avg(toFloat(r.smell)) as aroma
 order by FinalScore desc
 limit 1
""").data()

[{'style': 'New England IPA',
  'FinalScore': 8.797210377972094,
  'appearance': 4.383595613210904,
  'aroma': 4.41361476476119}]

## 9C. Which user is the most influential when it comes to reviews made?

To see which user is the most influential when it comes to reviews, a page rank algorithm was performed.

In [18]:
data = secure_graph.run(
    """
        CALL gds.graph.create(
            'group10-my-graph-question9c',
            [
                'Username',
                'Reviews'
            ],
            {
                MADE: {
                    orientation: 'REVERSE'
                }
            }
        )
    """
).data()
data

[{'nodeProjection': {'Username': {'properties': {}, 'label': 'Username'},
   'Reviews': {'properties': {}, 'label': 'Reviews'}},
  'relationshipProjection': {'MADE': {'orientation': 'REVERSE',
    'aggregation': 'DEFAULT',
    'type': 'MADE',
    'properties': {}}},
  'graphName': 'group10-my-graph-question9c',
  'nodeCount': 9238064,
  'relationshipCount': 9073128,
  'createMillis': 2010}]

In [14]:
data = secure_graph.run(
    """
        CALL gds.pageRank.stream('group10-my-graph-question9c') YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).user_name AS User, score
        ORDER BY score DESC LIMIT 1
    """
).data()
data

[{'User': 'Sammy', 'score': 1759.3714309692384}]

## 9D. Which beer styles are more central when it comes the amount of beers?

Similarly to the previous question, to find the most central beer styles, according to the amount of beers, a page rank algorithm was performed.

In [16]:
data = secure_graph.run(
    """
        CALL gds.graph.create(
            'group10-my-graph-question9d',
            [
                'Beers',
                'Style'
            ],
            {
                OF_TYPE: {
                    orientation: 'NATURAL'
                }
            }
        )
    """
).data()
data

[{'nodeProjection': {'Style': {'properties': {}, 'label': 'Style'},
   'Beers': {'properties': {}, 'label': 'Beers'}},
  'relationshipProjection': {'OF_TYPE': {'orientation': 'NATURAL',
    'aggregation': 'DEFAULT',
    'type': 'OF_TYPE',
    'properties': {}}},
  'graphName': 'group10-my-graph-question9d',
  'nodeCount': 358986,
  'relationshipCount': 358873,
  'createMillis': 4830}]

In [17]:
data = secure_graph.run(
    """
        CALL gds.pageRank.stream('group10-my-graph-question9d') YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS Style, score
        ORDER BY score DESC LIMIT 3
    """
).data()
data

[{'Style': 'American IPA', 'score': 5702.417230224609},
 {'Style': 'American Pale Ale (APA)', 'score': 2825.5490661621093},
 {'Style': 'American Imperial IPA', 'score': 2338.292272949219}]

## 10. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

We suggest the top 3 beers according to average review score, that have at least 500 reviews and are still being produced.

In [5]:
secure_graph.run("""
 match (r:Reviews)-[:ABOUT]-(b:Beers)
 with b, count(r) as review_numbers, avg(toFloat(r.score)) as avg_score
 where review_numbers > 500 and b.retired = 'f'
 return b.name AS beer, avg_score
 order by avg_score desc
 limit 3
""").data()

[{'beer': 'Kentucky Brunch Brand Stout', 'avg_score': 4.837735042735044},
 {'beer': 'Barrel-Aged Abraxas', 'avg_score': 4.742700831024935},
 {'beer': 'King Julius', 'avg_score': 4.736540600667414}]

## Submission of the results

In [1]:
import redis

r = redis.Redis(
    host='rhea.isegi.unl.pt',                            
    port='6388',                                         
    password='o307U52I854Oi6O2B893V9gv49051022', 
    decode_responses=True
)

In [2]:
r.set("0", "358873")
r.set("1", "200")
r.set("2.a", "IPA")
r.set("2.b", "Sierra Nevada Brewing Co.")
r.set("2.c", "US")
r.set("3", "acurtis")
r.set("4", "Dois Corvos Cervejeira")
r.set("5", "Finisterra")
r.set("6", "10.595992516144603")
r.set("7", "1648 Brewing Company Ltd")
r.set("8", "New England IPA")
r.set("9.c", "Sammy")
r.set("9.d", "American IPA")
r.set("10", "Kentucky Brunch Brand Stout, Barrel-Aged Abraxas, King Julius")

True